In [48]:
import os
from skimage import data, io, feature, util, transform
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

## Image Extraction

In [49]:
happy = len(os.listdir(f'happy/'))
sad = len(os.listdir(f'sad/'))

print(happy)
print(sad)

1536
1462


In [50]:
happy = os.listdir(f'happy/')
sad = os.listdir(f'sad/')

happy = ['happy/' + i for i in happy]
sad = ['sad/' + i for i in sad]

happy = np.array([io.imread(i, as_gray=True) for i in happy])
sad = np.array([io.imread(i, as_gray=True) for i in sad])

## Process Image to Dataframe

In [51]:
features = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']
angle = [0, 45, 90, 135, 180]

df = {}

for i in features:
    for j in angle:

        df[i.capitalize() + str(j)] = np.array([])

df['Label'] = np.array([])

for i in happy:
    
    for j in range(len(features)):

        a = feature.graycomatrix(i, distances=[1], angles=angle, levels=256,
                            symmetric=True, normed=True)
        a = feature.graycoprops(a, prop=features[j]).flatten()
        
        for k in range(len(angle)):

            df[features[j].capitalize() + str(angle[k])] = np.append(a[k], df[features[j].capitalize() + str(angle[k])])
    df['Label'] = np.append('Happy', df['Label'])

for i in sad:
    
    for j in range(len(features)):
        
        a = feature.graycomatrix(i, distances=[2], angles=angle, levels=256,
                            symmetric=True, normed=True)
        a = feature.graycoprops(a, prop=features[j]).flatten()
        
        for k in range(len(angle)):

            df[features[j].capitalize() + str(angle[k])] = np.append(a[k], df[features[j].capitalize() + str(angle[k])])
    df['Label'] = np.append('Sad', df['Label'])

df = pd.DataFrame(df)

## Make Preprocessing Function pt. 1

In [52]:
def preprocessing_1(photo_dir):
    
    from skimage import data, io, feature, util, transform
    import numpy as np
    
    features = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']
    angle = [0, 45, 90, 135, 180]

    res = np.array([])
    
    for j in features:
        
        a = feature.graycomatrix(photo_dir, distances=[2], angles=angle, levels=256,
                            symmetric=True, normed=True)
        a = feature.graycoprops(a, prop=features[j]).flatten()

        for k in range(len(angle)):

            res = np.append(a[k], res)
    
    return res

## Make Train and Test Data

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

X, y = df.drop(columns=['Label']), df['Label']
sc =  StandardScaler()
X = sc.fit_transform(X)
le = LabelEncoder()
y = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    random_state=0)

In [54]:
df[0:]

,Dissimilarity0,Dissimilarity45,Dissimilarity90,Dissimilarity135,Dissimilarity180,Correlation0,Correlation45,Correlation90,Correlation135,Correlation180,...,Asm45,Asm90,Asm135,Asm180,Energy0,Energy45,Energy90,Energy135,Energy180,Label
0,22.927083,30.656337,29.461610,22.927083,30.656337,0.781705,0.617046,0.640733,0.781705,0.617046,...,0.000324,0.000322,0.000354,0.000324,0.018816,0.018006,0.017941,0.018816,0.018006,Sad
1,11.473732,12.472710,12.368178,11.473732,12.472710,0.640341,0.597313,0.619094,0.640341,0.597313,...,0.000638,0.000603,0.000719,0.000638,0.026823,0.025269,0.024549,0.026823,0.025269,Sad
2,13.730525,14.517576,13.694265,13.730525,14.517576,0.767347,0.711087,0.742162,0.767347,0.711087,...,0.000445,0.000452,0.000436,0.000445,0.020875,0.021100,0.021262,0.020875,0.021100,Sad
3,28.532609,29.163737,27.483349,28.532609,29.163737,0.762508,0.726521,0.770769,0.762508,0.726521,...,0.000294,0.000301,0.000288,0.000294,0.016958,0.017151,0.017353,0.016958,0.017151,Sad
4,26.992754,32.218779,32.457447,26.992754,32.218779,0.762148,0.680280,0.668240,0.762148,0.680280,...,0.000326,0.000329,0.000325,0.000326,0.018034,0.018042,0.018125,0.018034,0.018042,Sad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2993,16.086436,18.437302,11.515957,16.086436,18.437302,0.884071,0.854282,0.941135,0.884071,0.854282,...,0.000364,0.000423,0.000392,0.000364,0.019801,0.019080,0.020560,0.019801,0.019080,Happy
2994,16.717199,19.872340,14.079344,16.717199,19.872340,0.859893,0.819672,0.906723,0.859893,0.819672,...,0.000346,0.000382,0.000369,0.000346,0.019214,0.018610,0.019539,0.019214,0.018610,Happy
2995,12.128989,17.239475,11.693706,12.128989,17.239475,0.919512,0.834681,0.912924,0.919512,0.834681,...,0.000520,0.000673,0.000627,0.000520,0.025034,0.022804,0.025941,0.025034,0.022804,Happy
2996,10.911348,13.297420,8.579787,10.911348,13.297420,0.952034,0.936929,0.973215,0.952034,0.936929,...,0.000457,0.000583,0.000560,0.000457,0.023658,0.021377,0.024147,0.023658,0.021377,Happy


## Make Neural Network Model

In [55]:
from tensorflow.keras import Sequential, layers

model = Sequential([
    layers.Dense(30, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(1024, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(2, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [56]:
model.fit(X_train, y_train, epochs=50)

Epoch 1/50


71/71 [==============================] - 6s 37ms/step - loss: 0.1432 - accuracy: 0.9480
Epoch 2/50
71/71 [==============================] - 3s 37ms/step - loss: 0.0811 - accuracy: 0.9715
Epoch 3/50
71/71 [==============================] - 3s 38ms/step - loss: 0.0467 - accuracy: 0.9769
Epoch 4/50
71/71 [==============================] - 3s 37ms/step - loss: 0.0501 - accuracy: 0.9831
Epoch 5/50
71/71 [==============================] - 3s 38ms/step - loss: 0.0349 - accuracy: 0.9853
Epoch 6/50
71/71 [==============================] - 3s 37ms/step - loss: 0.0290 - accuracy: 0.9884
Epoch 7/50
71/71 [==============================] - 3s 37ms/step - loss: 0.0195 - accuracy: 0.9929
Epoch 8/50
71/71 [==============================] - 3s 37ms/step - loss: 0.0208 - accuracy: 0.9911
Epoch 9/50
71/71 [==============================] - 3s 39ms/step - loss: 0.0200 - accuracy: 0.9933
Epoch 10/50
71/71 [==============================] - 3s 37ms/step - loss: 0.0321 - accuracy: 0.9911
Epoch 11/50
71/71 [=

## Evaluate The Model

In [57]:
model.evaluate(X_test, y_test)

24/24 [==============================] - 0s 6ms/step - loss: 0.1595 - accuracy: 0.9893


[0.15951824188232422, 0.9893333315849304]

In [58]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
y_pred = model.predict(X_test)
y_predi = y_pred[:, 0] < y_pred[:, 1]
for i in range(len(y_predi)):
    y_predi[i] = int(y_predi[i])
        
score = {'Accuracy':accuracy_score(y_test, y_predi),
         'Precision':precision_score(y_test, y_predi, average=None),
         'Recall':recall_score(y_test, y_predi, average=None),
         'F1':f1_score(y_test, y_predi, average=None),
         }
pd.DataFrame(score)

24/24 [==============================] - 0s 7ms/step


,Accuracy,Precision,Recall,F1
0,0.989333,0.988439,0.988439,0.988439
1,0.989333,0.990099,0.990099,0.990099


In [59]:
avg = {'Accuracy':[pd.DataFrame(score)['Accuracy'].mean()],
       'Precision':[pd.DataFrame(score)['Precision'].mean()],
       'Recall':[pd.DataFrame(score)['Recall'].mean()],
       'F1':[pd.DataFrame(score)['F1'].mean()]}

pd.DataFrame(avg)

,Accuracy,Precision,Recall,F1
0,0.989333,0.989269,0.989269,0.989269


In [60]:
X_train[0].shape

(30,)

## Save Preprocessing and Model into .pkl Extension

In [61]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 30)                930       
                                                                 
 dense_17 (Dense)            (None, 256)               7936      
                                                                 
 dense_18 (Dense)            (None, 512)               131584    
                                                                 
 dense_19 (Dense)            (None, 1024)              525312    
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0         
                                                                 
 dense_20 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_21 (Dense)            (None, 512)              

In [62]:
from pickle import dump
dump(model, open('model.pkl', 'wb'))
dump(sc, open('scaling.pkl', 'wb'))
dump(le, open('label.pkl', 'wb'))